In [ ]:
import torch
import os
import glob
import json
import numpy as np
from PIL import Image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision
from tqdm.auto import tqdm # tqdm 임포트

# =================================================================================
# 0. 콜백 클래스 정의 (EarlyStopping)
# =================================================================================
# 이 클래스는 훈련을 효율적으로 관리하는 전문가용 기능입니다.
class EarlyStopping:
    """검증 손실이 개선되지 않으면 훈련을 조기에 중단시킵니다."""
    def __init__(self, patience=5, min_delta=0, verbose=False):
        """
        Args:
            patience (int): 개선이 없다고 판단하기까지 기다릴 에폭 수.
            min_delta (float): 개선으로 인정할 최소 변화량.
            verbose (bool): 조기 중단 시 메시지를 출력할지 여부.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = np.inf
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                if self.verbose:
                    print(f"EarlyStopping: 검증 손실이 {self.patience} 에폭 동안 개선되지 않았습니다. 훈련을 중단합니다.")
                self.early_stop = True

# 0. 콜백 클래스 정의 (CheckpointSaver 추가)
class CheckpointSaver:
    """검증 손실을 기준으로 상위 K개의 모델만 저장하고 관리합니다."""
    def __init__(self, save_dir='checkpoints', top_k=3, verbose=False):
        """
        Args:
            save_dir (str): 체크포인트를 저장할 디렉토리.
            top_k (int): 유지할 상위 모델의 개수.
            verbose (bool): 모델 저장/삭제 시 메시지를 출력할지 여부.
        """
        self.save_dir = save_dir
        self.top_k = top_k
        self.verbose = verbose
        # (loss, filepath) 튜플을 저장할 리스트
        self.checkpoints = []
        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir)

    def __call__(self, val_loss, epoch, model):
        # 파일명 생성 (Keras 스타일)
        filename = f"EPOCH({epoch+1:02d})-LOSS({val_loss:.4f}).pth"
        filepath = os.path.join(self.save_dir, filename)

        # 현재 저장된 체크포인트가 K개 미만이거나,
        # 현재 손실이 저장된 체크포인트 중 가장 나쁜(가장 큰) 손실보다 좋을 때만 저장
        if len(self.checkpoints) < self.top_k or val_loss < self.checkpoints[-1][0]:
            # 모델 저장
            torch.save(model.state_dict(), filepath)
            
            # 리스트에 추가
            self.checkpoints.append((val_loss, filepath))
            
            # 손실을 기준으로 오름차순 정렬 (가장 좋은 모델이 맨 앞에 오도록)
            self.checkpoints.sort(key=lambda x: x[0])

            if self.verbose:
                print(f"  -> 체크포인트 저장: {filepath} (검증 손실: {val_loss:.4f})")

            # 만약 저장된 체크포인트가 K개를 초과하면, 가장 나쁜 모델 삭제
            if len(self.checkpoints) > self.top_k:
                worst_checkpoint = self.checkpoints.pop() # 가장 마지막 요소 (가장 나쁜 손실)
                try:
                    os.remove(worst_checkpoint[1])
                    if self.verbose:
                        print(f"  -> 오래된 체크포인트 삭제: {worst_checkpoint[1]}")
                except OSError as e:
                    print(f"Error removing file {worst_checkpoint[1]}: {e}")

# =================================================================================
# 1. 데이터셋 클래스 정의 (PillDataset)
# =================================================================================
class PillDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.annotation_paths = sorted(glob.glob(os.path.join(self.root, 'train_annotations', '**', '*.json'), recursive=True))
        
        self.categories = self._get_all_categories()
        self.cat_to_id = {cat['name']: cat['id'] for cat in self.categories}
        self.id_to_cat = {cat['id']: cat['name'] for cat in self.categories}
        
        self.class_ids = sorted(self.cat_to_id.values())
        self.map_cat_id_to_label = {cat_id: i + 1 for i, cat_id in enumerate(self.class_ids)}
        
        print(f"총 {len(self.annotation_paths)}개의 annotation 파일을 찾았습니다.")
        print(f"총 {len(self.class_ids)}개의 고유한 클래스를 발견했습니다.")

    def _get_all_categories(self):
        all_cats = {}
        # tqdm을 사용해 카테고리 로딩 진행 상황을 보여줍니다.
        for ann_path in tqdm(self.annotation_paths, desc="카테고리 정보 로딩 중"):
            with open(ann_path, 'r') as f:
                data = json.load(f)
                if 'categories' in data:
                    for cat in data['categories']:
                        if cat['id'] not in all_cats:
                            all_cats[cat['id']] = cat
        return list(all_cats.values())

    def __getitem__(self, idx):
        ann_path = self.annotation_paths[idx]
        
        with open(ann_path, 'r') as f:
            data = json.load(f)
        
        image_info = data['images'][0]
        img_path = os.path.join(self.root, 'train_images', image_info['file_name'])
        
        try:
            img = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            # Colab 환경에서는 파일을 못찾으면 다음으로 넘어가는게 중요
            return None 

        annotations = data['annotations']
        # 유효한 어노테이션이 없는 경우 건너뛰기
        if not annotations or not any(ann.get('bbox') for ann in annotations):
            return None

        boxes = []
        labels = []
        for ann in annotations:
            # bbox 정보가 없거나 유효하지 않은 어노테이션은 건너뜁니다.
            if 'bbox' not in ann or not ann['bbox']:
                continue
            x_min, y_min, w, h = ann['bbox']
            # bbox가 비정상적인 경우 건너뜁니다.
            if w <= 0 or h <= 0:
                continue
            x_max, y_max = x_min + w, y_min + h
            boxes.append([x_min, y_min, x_max, y_max])
            
            original_cat_id = ann['category_id']
            labels.append(self.map_cat_id_to_label[original_cat_id])

        # 유효한 박스가 하나도 없으면 이 데이터는 무시합니다.
        if not boxes:
            return None

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        # area, iscrowd 등 모델이 요구하는 다른 키들도 추가해줍니다.
        target["area"] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        target["iscrowd"] = torch.zeros((len(boxes),), dtype=torch.int64)


        if self.transforms:
            # torchvision v2 transform은 이미지와 타겟을 함께 받습니다.
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.annotation_paths)

# =================================================================================
# 2. 모델 인스턴스 생성 함수
# =================================================================================
def get_model_instance(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# =================================================================================
# 3. 데이터 변환 및 로더를 위한 헬퍼 함수
# =================================================================================
def get_transform(train):
    import torchvision.transforms.v2 as T
    transforms = []
    transforms.append(T.ToImage())
    transforms.append(T.ToDtype(torch.float32, scale=True))
    if train:
        # 훈련 시에만 데이터 증강을 추가합니다.
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

def collate_fn(batch):
    # 배치 내의 None 데이터를 걸러냅니다.
    batch = list(filter(lambda x: x is not None, batch))
    if not batch: # 만약 배치가 비어있으면 None을 반환
        return (None, None)
    return tuple(zip(*batch))

# =================================================================================
# 4. 메인 실행 블록
# =================================================================================
if __name__ == '__main__':
    # --- 설정 변수 ---
    # !!! Colab 사용 시, 이 경로를 Google Drive에 마운트된 경로로 변경하세요 !!!
    # 예: ROOT_DIRECTORY = "/content/drive/MyDrive/pill_project"
    # ROOT_DIRECTORY = "./" 
    ROOT_DIRECTORY = "/kaggle/input/train-pill" 
    DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    NUM_EPOCHS = 20 # Early Stopping을 테스트하기 위해 에폭 수를 늘립니다.
    BATCH_SIZE = 4
    
    print(f"사용할 장치: {DEVICE}")

    # --- 데이터셋 및 데이터로더 준비 ---
    # 훈련 데이터셋 (증강 적용)
    dataset_train = PillDataset(root=ROOT_DIRECTORY, transforms=get_transform(train=True))
    # 검증 데이터셋 (증강 미적용)
    dataset_valid = PillDataset(root=ROOT_DIRECTORY, transforms=get_transform(train=False))
    
    # 클래스 수 결정 (배경 포함)
    num_classes = len(dataset_train.class_ids) + 1
    
    # 훈련/검증 데이터셋 분할
    indices = torch.randperm(len(dataset_train)).tolist()
    train_size = int(len(dataset_train) * 0.8)
    train_subset = torch.utils.data.Subset(dataset_train, indices[:train_size])
    valid_subset = torch.utils.data.Subset(dataset_valid, indices[train_size:])

    data_loader_train = torch.utils.data.DataLoader(
        train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, collate_fn=collate_fn
    )
    data_loader_valid = torch.utils.data.DataLoader(
        valid_subset, batch_size=1, shuffle=False, num_workers=2, collate_fn=collate_fn
    )
    
    # --- 모델 및 옵티마이저 준비 ---
    model = get_model_instance(num_classes)
    model.to(DEVICE)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    
    # --- 콜백 및 변수 초기화 ---
    early_stopper = EarlyStopping(patience=5, verbose=True)
    best_val_loss = np.inf

    # CheckpointSaver 인스턴스 생성 (상위 3개 모델 저장)
    checkpoint_saver = CheckpointSaver(save_dir='/kaggle/working/checkpoints', top_k=3, verbose=True)

    # --- 훈련 및 검증 루프 ---
    for epoch in range(NUM_EPOCHS):
        # 훈련
        model.train()
        train_loss = 0.0
        # tqdm을 사용한 훈련 루프
        progress_bar_train = tqdm(data_loader_train, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [훈련]")
        for images, targets in progress_bar_train:
            # collate_fn에서 배치가 비어있을 수 있음
            if images is None:
                continue
            images = list(image.to(DEVICE) for image in images)
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            
            train_loss += losses.item()
            progress_bar_train.set_postfix(loss=losses.item()) # tqdm에 현재 손실 표시
        
        avg_train_loss = train_loss / len(data_loader_train)
        
        # 검증
        model.eval()
        val_loss = 0.0
        # tqdm을 사용한 검증 루프
        progress_bar_valid = tqdm(data_loader_valid, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [검증]")
        with torch.no_grad(): # 검증 시에는 그래디언트 계산 불필요
            for images, targets in progress_bar_valid:
                if images is None:
                    continue
                images = list(image.to(DEVICE) for image in images)
                targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
                
                # 검증 시에는 훈련 모드와 달리, 모델이 예측과 손실을 함께 반환하지 않을 수 있음.
                # 이를 위해 모델을 훈련 모드로 잠시 바꾸고, 타겟을 함께 넣어 손실을 계산합니다.
                model.train()
                loss_dict = model(images, targets)
                model.eval()

                losses = sum(loss for loss in loss_dict.values())
                val_loss += losses.item()
                progress_bar_valid.set_postfix(loss=losses.item())


        avg_val_loss = val_loss / len(data_loader_valid)
        print(f"Epoch {epoch+1}: 훈련 손실 = {avg_train_loss:.4f}, 검증 손실 = {avg_val_loss:.4f}")
        
        # # 최고 성능 모델 저장
        # if avg_val_loss < best_val_loss:
        #     best_val_loss = avg_val_loss
        #     torch.save(model.state_dict(), 'best_model.pth')
        #     print(f"  -> 새로운 최고 성능 모델 저장! (검증 손실: {best_val_loss:.4f})")
        
        # 최고 성능 모델 저장 로직을 CheckpointSaver 호출로 대체
        checkpoint_saver(avg_val_loss, epoch, model)

        # Early Stopping 체크
        early_stopper(avg_val_loss)
        if early_stopper.early_stop:
            break # 훈련 루프 탈출
        
        # 학습률 스케줄러 업데이트
        lr_scheduler.step()

    print("훈련 종료!")
    print(f"최고 검증 손실: {best_val_loss:.4f}")
    
    print("최종 저장된 상위 모델들:")
    for loss, path in checkpoint_saver.checkpoints:
        print(f"  - {path} (검증 손실: {loss:.4f})")


print("훈련이 모두 종료되었습니다.")
print(f"최고 검증 손실: {checkpoint_saver.checkpoints[0][0]:.4f}") # CheckpointSaver 사용 시
print(f"최고 성능 모델은 '{checkpoint_saver.checkpoints[0][1]}'에 저장되었습니다.")

# =================================================================================
# 5. mAP 평가 실행 블록
# =================================================================================
print("\n--- 최고 성능 모델로 mAP 평가를 시작합니다 ---")

# mAP 계산을 위한 라이브러리 설치 확인
try:
    from torchmetrics.detection.mean_ap import MeanAveragePrecision
except ImportError:
    print("torchmetrics 라이브러리가 설치되지 않았습니다. 설치를 시작합니다.")
    # Colab 환경에서 pip 설치 실행
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "torchmetrics"])
    from torchmetrics.detection.mean_ap import MeanAveragePrecision


# --- 최고 성능 모델 로드 ---
# CheckpointSaver가 저장한 가장 좋은 모델의 경로를 가져옵니다.
best_model_path = checkpoint_saver.checkpoints[0][1]
# 모델 구조를 다시 만들고, 저장된 가중치를 불러옵니다.
model.load_state_dict(torch.load(best_model_path))
model.to(DEVICE)


# --- mAP 계산기 및 평가 데이터로더 준비 ---
metric = MeanAveragePrecision(iou_type="bbox", iou_thresholds=[0.5])
# 평가에는 검증 데이터셋(valid_subset)을 사용합니다.
data_loader_eval = torch.utils.data.DataLoader(
    valid_subset, batch_size=4, shuffle=False, num_workers=2, collate_fn=collate_fn
)

model.eval() # 모델을 평가 모드로 설정
with torch.no_grad():
    progress_bar = tqdm(data_loader_eval, desc="mAP 계산 중")
    for images, targets in progress_bar:
        if images is None:
            continue
        
        images = list(image.to(DEVICE) for image in images)
        
        outputs = model(images)
        
        preds = []
        for output in outputs:
            preds.append({
                "boxes": output["boxes"].cpu(),
                "scores": output["scores"].cpu(),
                "labels": output["labels"].cpu(),
            })

        target_formatted = []
        for t in targets:
            target_formatted.append({
                "boxes": t["boxes"].cpu(),
                "labels": t["labels"].cpu(),
            })

        metric.update(preds, target_formatted)

# --- 최종 결과 출력 ---
results = metric.compute()
print("\n--- 최종 mAP@0.5 결과 ---")
print(f"  mAP: {results['map']:.4f}")
print(f"  mAP@.50 (대회 기준): {results['map_50']:.4f}") # IoU 0.50 에서의 mAP
print(f"  mAP (small): {results['map_small']:.4f}")
print(f"  mAP (medium): {results['map_medium']:.4f}")
print(f"  mAP (large): {results['map_large']:.4f}")